<a href="https://colab.research.google.com/github/helderthadeu/face-recognition/blob/main/Deteccao_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [3]:


# Carregar o ResNet50 sem as camadas de classificação superiores
# Adicionamos `include_top=False` e definimos o `input_shape`
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [4]:
# Congelar as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Adicionar novas camadas para o nosso problema
num_classes = 3 # Número de classes
x = base_model.output
x = GlobalAveragePooling2D()(x) # Converte os mapas de características para um vetor
x = Dense(1024, activation='relu')(x) # Camada densa com 1024 neurônios
predictions = Dense(num_classes, activation='softmax')(x) # Camada de saída com o número de classes (faces)

# Criar o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
# Substitua 'num_classes' pelo número de pessoas no seu dataset
num_classes = 3  # Exemplo: Se você tiver 3 pessoas

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Preparar o gerador de dados (certifique-se de que as imagens estão na estrutura de pastas correta)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Files/FotosTreinamento',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Files/FotosValidation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Treinar o modelo
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Found 1986 images belonging to 3 classes.
Found 288 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/10
30/63 ━━━━━━━━━━━━━━━━━━━━ 4:04 7s/step - accuracy: 0.7733 - loss: 0.9469